In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import norm

# Chapter 1: Introduction to Data

# Chapter 2: Summarizing Data

# Chapter 3: Probability

# Chapter 4: Distributions of Random Variables

In [8]:
# probability of finding first success in 
# Nth trial if
# P is probability of success and
# 1-P is probability of failure
def geometric_distribution(p:float, n:int):
    from math import sqrt

    probability = p * (1-p)**(n-1)
    mean = 1 / p
    variance = (1-p)/(p**2)
    std = math.sqrt(variance)

    return probability, mean, variance, std

In [10]:
# GP 4.23
print(round(1/0.7, 2))

1.43


In [14]:
prob = 0.7
p_list = []
for i in range(1, 4):
    p, mu, sigma, std = geometric_distribution(prob, i)
    p_list.append(p)
print(sum(p_list))
p_complement = (1-prob)**3
print(1-p_complement)

0.973
0.973
